# Results report

This notebook summarizes the results obtained from the current model versus the baseline providing quantitative and qualitative results

## Comparison with the baseline [Uncertainty-Aware CNNs for Depth Completion: Uncertainty from Beginning to End](https://arxiv.org/abs/2006.03349)

### Area Under the Curve

In [ ]:
import torch
from lidar_confidence.metrics import percentiles
from lidar_confidence.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import functional as F
import sys
import os
from pathlib import Path
import warnings
from tqdm import tqdm
import json
from tabulate import tabulate
from collections import defaultdict
import matplotlib.pyplot as plt
import glob
from PIL import Image
import numpy as np

sys.path.append(str(Path("../experiments/baselines/ncnn/models")))

In [ ]:
with open(Path("../results/lce/lce.json"), "rt") as f:
    model_results = json.load(f)

ncnn_baselines_names = [
    "ncnn_conf_l1",
    "ncnn_conf_l1_kitti_limited",
    "ncnn_conf_l2",
    "ncnn_conf_l2_kitti_limited",
    "pncnn_exp",
    "pncnn_exp_kitti_limited",
    "pncnn",
    "pncnn_kitti_limited",
]
    
ncnn_baselines = {}
for file_path in [f"../results/baselines/{name}.json" for name in ncnn_baselines_names]:
    with open(Path(file_path), "rt") as f:
        name = Path(file_path).name.split(".")[0]
        ncnn_baselines[name] = json.load(f)   

headers = ["name", "test_1/auc_mae", "test_1/auc_rmse", "test_2/auc_mae", "test_2/auc_rmse"]
lines = []
lines.append(["ours", *[model_results[f] for f in headers[1:]]])
for name in ncnn_baselines:
    lines.append([name, *[ncnn_baselines[name][f] for f in headers[1:]]])

print("AUC METRICS\n")
print(tabulate(lines, headers=headers))

### RMSE removing the less confident measures

In [ ]:
def transform(d):
    for k in d:
        d[k] = F.to_tensor(d[k])
    d["img"] = F.normalize(d["img"], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    return d

ds_split_142 = Dataset("../data/dataset", transform=transform, split="test_2")
dl_split_142 = DataLoader(ds_split_142)

model = torch.jit.load("../results/lce/model.pth").to("cuda")

baselines = {}
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for baseline_name in ncnn_baselines_names:
        sys.path.append(str(Path(f"../experiments/baselines/ncnn/models/{baseline_name}")))
        baselines[baseline_name] = (
            torch.load(f"../experiments/baselines/ncnn/models/{baseline_name}/model.pth", map_location="cuda")["model"]
        )

In [ ]:
thresholds = [100, 95, 90, 80, 70, 60, 50, 40, 30, 20]

def compute_error(lidar, gt, conf_mask, depth_mask=None):
    
    if depth_mask is None:
        mask = (lidar > 0) & (gt > 0)
    else:
        mask = depth_mask
        
    lidar_, gt_ = lidar[mask][conf_mask], gt[mask][conf_mask]
    err = torch.sqrt(torch.mean(torch.square(lidar_ - gt_)))
    return err

errs = defaultdict(lambda: [])
for batch in tqdm(dl_split_142):
    mask = (batch["lidar"] > 0) & (batch["gt"] > 0)
    
    with torch.no_grad():
        std = model(torch.cat([batch["img"].to("cuda"), batch["lidar"].to("cuda")], 1)).cpu()[mask]
        
        base_confs = {}
        for name, base_model in baselines.items():
            base_confs[name] = base_model(batch["lidar"].to("cuda"))[:, 2:].cpu()[mask]
        
    mine_err = []
    for perc in thresholds:
        percentile = percentiles(std, [perc])[0, 0]
        mine_err.append(compute_error(batch["lidar"], batch["gt"], std <= percentile))
    errs["our"].append(mine_err)
    
    for name in baselines:
        base_err = []
        for perc in thresholds:
            percentile = percentiles(base_confs[name], [100 - perc])[0, 0]
            base_err.append(compute_error(batch["lidar"], batch["gt"], base_confs[name] >= percentile))
        errs[name].append(base_err)
        
lines = []
for key in errs:
    lines.append([key, *map(lambda x: x.item(), torch.tensor(errs[key]).mean(axis=0))])

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("RMSE progressively removing points with the worst confidence")
plt.gca().set_xlim(left=101, right=19)
plt.gca().set_xlabel("percentile (%)")
plt.gca().set_ylabel("error (m)")

for line in lines:
    plt.plot(thresholds, line[1:], label=line[0], markersize=10, marker=".")

plt.legend()
plt.show()

And finally below an image of the points removed by our method

In [ ]:
dl_iter = iter(dl_split_142)
for i in range(1):
    batch = next(dl_iter)
    
def depth_imshow(depth):
    depth = depth.clone()
    depth[depth > 0] = depth.max() - depth[depth > 0]
    plt.imshow(depth, cmap="inferno")

def normalize(img):
    return (img - img.min()) / (img.max() - img.min())
    
mask = batch["lidar"] > 0
with torch.no_grad():
    std = model(torch.cat([batch["img"].to("cuda"), batch["lidar"].to("cuda")], 1)).cpu()
    percentile = percentiles(std[mask], [85])[0, 0]
    mask_ = mask & (std <= percentile)

plt.figure(figsize=(20, 20))
plt.subplot(3, 1, 1); plt.title("image"); plt.imshow(normalize(batch["img"])[0].permute(1, 2, 0))
plt.subplot(3, 1, 2); plt.title("lidar points"); depth_imshow(batch["lidar"][0, 0])
plt.subplot(3, 1, 3); plt.title("lidar filtered"); depth_imshow(torch.where(
    mask_, batch["lidar"], torch.tensor(0.)
)[0, 0])
plt.show()

## Comparison with [A Surface Geometry Model for LiDAR Depth Completion](https://arxiv.org/pdf/2104.08466.pdf)

This paper provides a way to compute a binary mask to remove outliers thus AUC based metrics are not useful

In [ ]:
def compute_error(lidar, gt, conf_mask, depth_mask=None):
    
    if depth_mask is None:
        mask = (lidar > 0) & (gt > 0)
    else:
        mask = depth_mask
        
    lidar_, gt_ = lidar[mask][conf_mask], gt[mask][conf_mask]
    err = torch.sqrt(torch.mean(torch.square(lidar_ - gt_)))
    return err

# prepare data

def transform(d):
    for k in d:
        d[k] = F.to_tensor(d[k])
    d["img"] = F.normalize(d["img"], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    return d

class DatasetWithMasks(torch.utils.data.Dataset):
    
    def __init__(self, split):
        self.ds = Dataset("../data/dataset", split=split)
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, x):
        batch = self.ds[x]
        
        idx = str(x).zfill(10)
        batch["baseline_mask"] = np.array(
            Image.open(Path(f"../experiments/baselines/surface_geometry_model/{split}_masks/{idx}.png"))
        )[..., None]
        batch["baseline_mask"] = batch["baseline_mask"] / 256
        batch["baseline_mask"] = batch["baseline_mask"].astype(bool)
        
        return transform(batch)
    
for split in ["test_1", "test_2"]:
    print(f"\n== on {split} == ")
    ds = DatasetWithMasks(split)
    dl = DataLoader(ds)

    # compute the error
    error_100 = []
    error_filtered = []
    removed_perc = []
    for batch in dl:
        mask = (batch["lidar"] > 0) & (batch["gt"] > 0)
        error_100.append(torch.sqrt(torch.mean(torch.square(batch["lidar"][mask] - batch["gt"][mask]))))

        orig_mask_num = mask.sum()
        mask = mask & batch["baseline_mask"]
        filtered_mask_num = mask.sum()
        removed_perc.append(filtered_mask_num / orig_mask_num)

        error_filtered.append(torch.sqrt(torch.mean(torch.square(batch["lidar"][mask] - batch["gt"][mask]))))

    error_100 = torch.stack(error_100).mean().item()
    error_filtered = torch.stack(error_filtered).mean().item()
    removed_perc = 100 - torch.stack(removed_perc).mean().item() * 100
    print("Error before filtering: {:3.5f} m".format(error_100))
    print("Error after filtering:  {:3.5f} m".format(error_filtered))
    print("% of removed measures:  {:3.5f} %".format(removed_perc))

    error = []
    for batch in dl:
        mask = (batch["lidar"] > 0) & (batch["gt"] > 0)

        with torch.no_grad():
            std = model(torch.cat([batch["img"].to("cuda"), batch["lidar"].to("cuda")], 1)).cpu()[mask]

        percentile = percentiles(std, [100 - removed_perc])[0, 0]
        error.append(compute_error(batch["lidar"], batch["gt"], std <= percentile))

    error = torch.stack(error).mean().item()
    print("\nOur error removing {:3.5f} %: {:3.5f} m".format(removed_perc, error))